In [125]:
from mesa import Model
from mesa import Agent
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random
import numpy as np

from a_star_pathfinding import a_star
from vision import lines
from vision import transform

# Tijdelijke fix voor a_star:
def a_star(start, finish, grid):
    
    finished = False
    open_set = {start}
    closed_set = set()
    path = {}
    
    #function to calculate the distance between a node and the finish in order to calculate the costs
    def distance(node1, node2):
        x, y = node1
        x_finish, y_finish = node2
        dist = ((x - x_finish)**2 + (y - y_finish)**2)**.5
        return dist
    
    #this score is the absolute score from the start point up untill the current point
    score_tobegin = {}
    score_tobegin[start] = 0
    
    #guess scores scores the estimater score from a point to the finish
    guess_scores = {}
    guess_scores[start] = distance(start, finish)
    
    
    while open_set:
        
        guess = 1000000
        #the current node is the lowest scoring guess point, which is in the guess_scores
        for node in open_set:
            if guess_scores[node] < guess:
                guess = guess_scores[node]
                bestnode = node
        
        current_node = bestnode
        
        #find neighbors of current best node and check which step is the best improvement
        neighbors = grid.get_neighborhood(current_node, True, False, 1)
        
        open_set.remove(current_node)
        closed_set.add(current_node)
        for neighbor in neighbors:
            
            #first check of this step not an obstacle SO CHANGE THE FALSE STATEMENT TO CHECK FOR OBSTACLE

            if not grid.is_cell_empty(neighbor) and not type(grid.get_cell_list_contents(neighbor)[0]) == Vent:
                closed_set.add(neighbor)
                continue
                
            #check if the step is in the closed list
            if neighbor in closed_set:
                continue
            
            #calculate the cost of travelling to the neighbor from the begin and estimate cost to the finish
            score_from_begin = score_tobegin[current_node] + 1
            score_to_finish = distance(neighbor, finish)
            total_score = score_from_begin + score_to_finish
            
            #check if the point is already int the open set, if so check if the path is slower, if so continue with another neighbor
            if neighbor in open_set:
                if score_tobegin[neighbor] < score_from_begin:
                    continue
                
            open_set.add(neighbor)
            score_tobegin[neighbor] = score_from_begin
            guess_scores[neighbor] = total_score
            path[neighbor] = current_node
            
            if neighbor == finish:
                open_set = {}
                final_score = total_score
                break
            
    final_path = [finish]
    current = finish
    while current != start:
        current = path[current]
        final_path.append(current)
    
    final_path.pop(-1)

    return final_path, final_score

In [126]:
def euclidean(pos_a, pos_b):
    return ((pos_a[0] - pos_b[0])**2 + (pos_a[1] - pos_b[1])**2)**.5

In [127]:
class AmongUs(Model):
    def __init__(self, map_name, n_crew, n_impo):
        super().__init__()

        # generate grid
        self.height = 138
        self.width = 242
        self.grid = MultiGrid(self.width, self.height, torus=True)

        # vents initialization
        self.vents_dict = {
            (125,110): (130, 110),
            (130,110): (125, 110),
            (115,110): (140, 130),
            (140,130): (115, 110)
            }
        self.vents = []

        # generate map
        self.generate_map(map_name)

        # create schedulers for agents
        self.schedule_Crewmate = RandomActivation(self)
        self.schedule_Imposter = RandomActivation(self)
        
        # initialize agents
        self.n_crew = n_crew
        self.n_impo = n_impo
        self.spawn_players()

    def generate_map(self, map_name):
        hard_walls = np.load(f'{map_name}/hardwalls.npy')
        
        for coord in hard_walls:
            coord[1] = self.height - 1 - coord[1]
            self.new_agent(Wall, tuple(coord))

        for coord in self.vents_dict:
            self.new_agent(Vent, tuple(coord))

    def spawn_players(self):
        # generate crewmates
        for _ in range(self.n_crew):
            self.new_agent(Crewmate, (125, 100))

        # generate imposters
        for _ in range(self.n_impo):
            self.new_agent(Imposter, (125, 100))

    def new_agent(self, agent_type, pos):
        '''
        Method that creates a new agent, and adds it to the correct scheduler.
        '''
        agent = agent_type(self.next_id(), self, pos)

        self.grid.place_agent(agent, pos)

        if agent_type == Crewmate or agent_type == Imposter:
            getattr(self, f'schedule_{agent_type.__name__}').add(agent)

        if agent_type == Vent:
            self.vents.append(agent)
        
    def step(self):
        '''
        Method that steps every agent. 
        
        Prevents applying step on new agents by creating a local list.
        '''

        self.schedule_Crewmate.step()
        self.schedule_Imposter.step()

        # self.datacollector.collect(self)

    def play_match(self, step_count=50):
        '''
        Method that runs the model for a specific amount of steps.
        '''
        for i in range(step_count):
            print(i)
            self.step()


In [128]:
class Crewmate(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos
        self.path = []
        self.goal = (90, 40)

    def find_goal(self):
        self.path = a_star(self.pos, self.goal, self.model.grid)[0]

    def step(self):
        if self.path == []:
            self.find_goal()
        
        grid = self.model.grid
        neighborhood = grid.get_neighborhood(self.pos, True)
        
        neighbors = self.model.grid.get_neighbors(self.pos, True)

        if len(neighbors) > 0:
            for neighbor in neighbors:
                if type(neighbor) == Wall and neighbor.pos in neighborhood:
                    neighborhood.remove(neighbor.pos)

        # random movement
        # grid.move_agent(self, random.choice(neighborhood))

        if self.path != []:
            self.model.grid.move_agent(self, self.path.pop())
            
        self.find_vision()
        
    def find_vision(self):
        '''
        Determines the vision of the agent 
        '''
        grid = self.model.grid
        origin_lines = lines
        transformed_lines = transform(origin_lines, self.pos)
        vision_neighborhood = []
        
        for i in range(len(transformed_lines)):
            found_wall = False
            for j in range(len(transformed_lines[i])):  
                agents_on_tile = grid.get_neighbors(pos=transformed_lines[i][j], moore=True, radius=0, include_center=True)

                # stop line if it bumps into a wall  
                for agent in agents_on_tile:
                    if type(agent) == Wall:
                        found_wall = True
                
                # append to vision neigborhood when no wall is found and if coordinate is not already appended before
                if found_wall == False:
                    if transformed_lines[i][j] not in vision_neighborhood:
                        vision_neighborhood.append(transformed_lines[i][j])  
                        

class Imposter(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos
        self.path = []
        self.goal = (90, 40)
        self.vents_enabled = True

        self.find_path()
    
    def find_path(self):
        best_path, best_score = a_star(self.pos, self.goal, self.model.grid)

        if self.vents_enabled:

            for vent in self.model.vents:
                vent_in = vent.pos_a
                vent_out = vent.pos_b
                # guess_vent_path = ((self.pos - vent_in)**2 + (vent_out - self.goal)**2)**.5
                guess_vent_path = euclidean(self.pos, vent_in) + euclidean(vent_out, self.goal)

                if guess_vent_path < best_score:
                    vent_path_in, vent_score_in = a_star(self.pos, vent_in, self.model.grid)
                    vent_path_out, vent_score_out = a_star(vent_out, self.goal, self.model.grid)
                    vent_path = vent_path_in + vent_path_out
                    vent_score = vent_score_in + vent_score_out
 
                    if vent_score < best_score:
                        best_score = vent_score
                        best_path = vent_path

        self.path = best_path

        def step(self):
            if self.path == []:
                self.find_goal()

            grid = self.model.grid
            neighborhood = grid.get_neighborhood(self.pos, True)
            
            neighbors = self.model.grid.get_neighbors(self.pos, True)

            if len(neighbors) > 0:
                for neighbor in neighbors:
                    if type(neighbor) == Wall and neighbor.pos in neighborhood:
                        neighborhood.remove(neighbor.pos)

        # random movement
        # grid.move_agent(self, random.choice(neighborhood))

            if self.path != []:
                self.model.grid.move_agent(self, self.path.pop())
            
            self.find_vision()
        
    def find_vision(self):
        '''
        Determines the vision of the agent 
        '''
        grid = self.model.grid
        origin_lines = lines
        transformed_lines = transform(origin_lines, self.pos)
        vision_neighborhood = []
        
        for i in range(len(transformed_lines)):
            found_wall = False
            for j in range(len(transformed_lines[i])):  
                agents_on_tile = grid.get_neighbors(pos=transformed_lines[i][j], moore=True, radius=0, include_center=True)

                # stop line if it bumps into a wall  
                for agent in agents_on_tile:
                    if type(agent) == Wall:
                        found_wall = True
                
                # append to vision neigborhood when no wall is found and if coordinate is not already appended before
                if found_wall == False:
                    if transformed_lines[i][j] not in vision_neighborhood:
                        vision_neighborhood.append(transformed_lines[i][j])  
                        
                        
class Wall(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

class Obstruction(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

class Vent(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos_a = pos
        self.pos_b = self.model.vents_dict[pos]

In [129]:
amongUs = AmongUs('the_skeld', 3, 1)

amongUs.play_match()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
